In [1]:
import os
import pickle

# edges are encoded at 8 bit bytes. The first bit encodes flipped or not. The last 4 bits encode slice (0, 1, 2) and number within the slice (0, 1, 2, 3)

edges   = [0x10, 0x11, 0x12, 0x13, 0x04, 0x05, 0x06, 0x07, 0x08, 0x09, 0x0A, 0x0B]
corners = [0x00, 0x01, 0x02, 0x03, 0x04, 0x05, 0x06, 0x07]
parity  = [0x00]
cube    = bytes(edges + corners + parity)


In [2]:
def generate_new(members, generators, subgroup_mask):
    
    members_list = set([bytes([membercube[i] & subgroup_mask[i] for i in range(0,21)]) for membercube in members.values()])

    candidates, new_members = dict(), dict()
    
    for name, membercube in members.items():
        for n, generator in generators.items():           
            new_name  = name + n
            new_twist_array = ([membercube[0x0F & generator[i]] ^ (generator[i] & 0xF0)  for i in range(0,12)] +
                              [(membercube[12 + (0x0F & generator[j])] & 0x0F) + (((membercube[12 + (0x0F & generator[j])] & 0xF0) + (generator[j] & 0xF0))%0xC0) for j in range(12, 20)] + 
                              [cube[20] ^ generator[20] ])
            new_twist = bytes(new_twist_array)
            
            twist_masked = bytes([new_twist[i] & subgroup_mask[i] for i in range(0,21)]) 
            
            if twist_masked not in members_list:
                new_members[new_name] = new_twist
                members_list.add(twist_masked)         
   
    return new_members
    
    

In [3]:
            ## Define base permutations 
moves = {  

'R' :bytes(b'\x00\x01\x07\x06\x04\x05\x02\x03\x08\t\n\x0b\x00\x01\x07\x06\x04\x05\x02\x03\x80'),
'L' :bytes(b'\x04\x05\x02\x03\x01\x00\x06\x07\x08\t\n\x0b\x05\x04\x02\x03\x00\x01\x06\x07\x80'),
'F' :bytes(b'\x00\x01\x02\x03\x04\t\x08\x07\x05\x06\n\x0b\x00\x86\x02\x84A\x05C\x07\x80'),
'B' :bytes(b'\x00\x01\x02\x03\x0b\x05\x06\n\x08\t\x04\x07\x87\x01\x85\x03\x04@\x06B\x80'),
'U' :bytes(b'\x88\x01\x02\x8b\x04\x05\x06\x07\x83\t\n\x80D\x01\x02G\x83\x05\x06\x80\x80'),
'D' :bytes(b'\x00\x8a\x89\x03\x04\x05\x06\x07\x08\x81\x82\x0b\x00EF\x03\x04\x82\x81\x07\x80'),
'S' :bytes(b'\x00\x01\x03\x02\x04\x05\x07\x06\x08\t\n\x0b\x00\x01\x03\x02\x04\x05\x07\x06\x00'),
'M' :bytes(b'\x01\x00\x02\x03\x05\x04\x06\x07\x08\t\n\x0b\x01\x00\x02\x03\x05\x04\x06\x07\x00'),
'G' :bytes(b'\x00\x01\x02\x03\x04\x06\x05\x07\t\x08\n\x0b\x00\x03\x02\x01\x06\x05\x04\x07\x00'),
'C' :bytes(b'\x00\x01\x02\x03\x07\x05\x06\x04\x08\t\x0b\n\x02\x01\x00\x03\x04\x07\x06\x05\x00'),
'V' :bytes(b'\x03\x01\x02\x00\x04\x05\x06\x07\x0b\t\n\x08\x03\x01\x02\x00\x07\x05\x06\x04\x00'),
'E' :bytes(b'\x00\x02\x01\x03\x04\x05\x06\x07\x08\n\t\x0b\x00\x02\x01\x03\x04\x06\x05\x07\x00'),
'r' :bytes(b'\x00\x01\x06\x07\x04\x05\x03\x02\x08\t\n\x0b\x00\x01\x06\x07\x04\x05\x03\x02\x80'),
'l' :bytes(b'\x05\x04\x02\x03\x00\x01\x06\x07\x08\t\n\x0b\x04\x05\x02\x03\x01\x00\x06\x07\x80'),
'f' :bytes(b'\x00\x01\x02\x03\x04\x08\t\x07\x06\x05\n\x0b\x00\x84\x02\x86C\x05A\x07\x80'),
'b' :bytes(b'\x00\x01\x02\x03\n\x05\x06\x0b\x08\t\x07\x04\x85\x01\x87\x03\x04B\x06@\x80'),
'u' :bytes(b'\x8b\x01\x02\x88\x04\x05\x06\x07\x80\t\n\x83G\x01\x02D\x80\x05\x06\x83\x80'),
'd' :bytes(b'\x00\x89\x8a\x03\x04\x05\x06\x07\x08\x82\x81\x0b\x00FE\x03\x04\x81\x82\x07\x80')

}





In [4]:
group_moves = ['RLFBUDrlfbudSMGCVE',  'RLFBrlfbSMGCVE', 'RLrlSMGCVE', 'SMGCVE' ]
group_masks = [[[0x80], [0x00], [0x00]], [[0x10], [0xC0], [0x00]], [[0x0C], [0x06], [0x8F]], [[0x0FF], [0xFF], [0x00]]]

thistle_tables = []

for group in range(0,4):
    move_dict = {move_key: moves[move_key] for move_key in group_moves[group]}
    bit_mask  = bytes(group_masks[group][0] * 12 + group_masks[group][1] * 8 + group_masks[group][2])
    new, members = 1, {'': cube}
    while new > 0 :
        new_members = generate_new(members, move_dict, bit_mask)
        new = len(new_members)
        members.update(new_members)
    print(group, len(members))
    thistle_tables.append(members)
   

0 2048
1 1082565
2 352800
3 663552


In [5]:
with open('thistle_tables.dat', 'wb') as file:
    pickle.dump(thistle_tables, file)

SyntaxError: invalid syntax (464334863.py, line 1)